In [4]:
# Load the environment
# import Pkg
# Pkg.activate("./NotebookEnv")

  Activating project at `~/Developer/research/summer2023/gridtasks2D/GridWorlds`


In [1]:
# Visualization Library
import WGLMakie

using Revise
using Gen

# Simple gridworld functionality
import GridWorlds

# Beginnings of a Gen POMDP library
import GenPOMDPs

LoadError: ArgumentError: Package GridWorlds not found in current path.
- Run `import Pkg; Pkg.add("GridWorlds")` to install the GridWorlds package.

In [ ]:
# Position prior

@gen (static) function uniform_agent_pos(params)
    w = params.map # a map, represented as a GridWorlds.GridWorld
    
    pos ~ uniform_from_set(GridWorlds.empty_cells(w))
    return pos
end

In [ ]:
# Motion model

@dist list_categorical(list, probs) = list[categorical(probs)]

function _get_motion_logprobs(w, (x2, y2), σ)
    possible_positions = reshape(Tuple.(keys(w)), (:,))
    logprobs = [
        w[x, y] == empty ? logpdf(broadcasted_normal, [x, y], [x2, y2], σ) : -Inf
        for (x, y) in possible_positions
    ]
    return (possible_positions, logprobs)
end
_normalized_probs(lps) = exp.(lps .- logsumexp(lps))

# action `a` should be a symbol in [:up, :down, :left, :right, :stay]
@gen (static) function motion_model(pos, action, params)
    w, σ = params.map, params.step.σ
    
    new_agent_position_deterministic = GridWorlds.newpos(w, pos, action)
    (possible_positions, logprobs) = _get_motion_logprobs(
        w, new_agent_position_deterministic, σ
    )
    pos ~ list_categorical(
        possible_positions,
        _normalized_probs(logprobs)
    )
    return pos
end

In [ ]:
# Observation model: LIDAR distance measurements along `n_rays`
# evenly spaced angles, with gaussian noise on the observed distances
@gen (static) function observe_noisy_distances(pos, params)
    w, n_rays, σ = params.map, params.obs.n_rays, params.obs.σ
    
    dists = ray_trace_distances(moveagent(w, pos), n_rays)
    obs ~ broadcasted_normal(dists, σ)
    return obs
end

In [ ]:
# POMDP of this environment
pomdp = GenPOMDPs.GenPOMDP(
    uniform_agent_pos,       # INIT   : params                      ⇝ state
    motion_model,            # STEP   : prev_state, actions, params ⇝ state
    observe_noisy_distances, # OBS    : state                       ⇝ observation
    (state, action) -> 0.    # UTILITY: state, action, params       → utility
)

In [ ]:
# Generative function over trajectories of the POMDP,
# given a fixed action sequence.
trajectory_model = GenPOMDPs.ControlledTrajectoryModel(pomdp)

In [ ]:
PARAMS = (
    # load houseexpo env #7,
    # discretized with 24 gridsquares along the x axis
    map = load_gridworld(24, 7),
    
    obs  = (; n_rays = 20, σ = 1.),
    step = (; σ = 0.1)
);

In [ ]:
# Set up "interactive mode" with this model.

# `take_action` is a function which accepts an action
# as input, and updates the `world_trace` by taking that action.

# `world_trace` is an `Observable` whose value is a trace of
# `trajectory_model`, describing the "ground truth" trajectory which
# has played out in the environment.  (This includes the state,
# action, observation, and utility sequences.)

# Each time `take_action` is called, `world_trace` automatically updates
# to extend the simulation roll-out by one timestep.

world_trace, take_action = GenPOMDPs.interactive_world_trace(trajectory_model, PARAMS);

In [ ]:
# Map the world_trace observable to an observable
# over the sequence of ground truth positions, and observation
# point clouds
function get_posobs_seq(groundtruth_trace)
    return Makie.@lift((
        GenPOMDPs.state_sequence($groundtruth_trace),
        GenPOMDPs.observation_retval_sequence($groundtruth_trace)
    ))
end

# Display this in an interactive GUI
(f, t) = GridWorlds.Viz.interactive_gui(
    PARAMS.map, get_posobs_seq(world_trace), take_action
)
f

In [ ]:
# New interactive session.
world_trace2, take_action2 = GenPOMDPs.interactive_world_trace(trajectory_model, PARAMS)

# Get the sequence of observations and actions from the
# world (consumed by inference).
function get_actobs_seq(groundtruth_trace)
    return Makie.@lift((
        GenPOMDPs.observation_sequence($groundtruth_trace),
        GenPOMDPs.action_sequence($groundtruth_trace)
    ))
end

# Maintain a 200-particle bootstrap particle filter,
# which updates whenever `world_trace2` updates
bootstrap_pf_states = GenPOMDPs.pf_observable(
    GenPOMDPs.bootstrap_pf(
        pomdp,  # Run inference in the ground truth POMDP
        PARAMS, # with the ground-truth parameters
        200     # using 200 particles
    ),
    get_actobs_seq(world_trace2)
);

# Display an interactive GUI, with the particles
(f2, t2) = interactive_mode_gui(PARAMS.map, get_posobs_seq(world_trace2), take_action2)
display_pf_localization!(f2, t2, bootstrap_pf_states)
f2

In [ ]:
# Same as before, but now with a better particle filter
world_trace3, take_action3 = GenPOMDPs.interactive_world_trace(trajectory_model, PARAMS)

function initially_stratified_pf(pomdp, params)
    possible_locations = empty_cells(params.map)
    return GenPOMDPs.pf(pomdp, params,
        ( # Stratified initial proposal
             ( # Strata = enumerate each possible position
                choicemap((GenPOMDPs.state_addr(0, :pos), loc))
                for loc in possible_locations
             ),
             length(possible_locations), # N particles
        ),
        () # bootstrap update after the first step
    )
end
stratified_pf_states = GenPOMDPs.pf_observable(
    initially_stratified_pf(pomdp, PARAMS),
    get_actobs_seq(world_trace3)
);

(f3, t3) = interactive_mode_gui(PARAMS.map, get_posobs_seq(world_trace3), take_action3)
display_pf_localization!(f3, t3, stratified_pf_states)
f3

In [ ]:
# Same as before, but in an environment where there can
# be genuine position uncertainty over a long period of time.

PARAMS_AMBIGUOUS = (
    # load a custom map I wrote with 2 identical rooms
    map = load_custom_map(1),
    
    # Very low observation noise
    obs  = (; n_rays = 20, σ = .0001),
    step = (; σ = 0.1)
);

world_trace4, take_action4 = GenPOMDPs.interactive_world_trace(trajectory_model, PARAMS_AMBIGUOUS)

stratified_pf_states4 = GenPOMDPs.pf_observable(
    initially_stratified_pf(pomdp, PARAMS_AMBIGUOUS),
    get_actobs_seq(world_trace4)
);

(f4, t4) = interactive_mode_gui(PARAMS_AMBIGUOUS.map, get_posobs_seq(world_trace4), take_action4)
display_pf_localization!(f4, t4, stratified_pf_states4)
f4